<a href="https://colab.research.google.com/github/lalroshan590/.github/blob/master/Copy_of_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DABstep Benchmark Qwen2.5-Coder-32B Baseline

This notebook will guide you though submitting a Qwen2.5-Coder-32B baseline to the benchmark.

* Live 🤗 Leaderboard: https://huggingface.co/spaces/adyen/DABstep
* Benchmark 🤗 Dataset: https://huggingface.co/datasets/adyen/DABstep
* LLM Agent Framework by 🤗: https://github.com/huggingface/smolagents/tree/main



## Environment Setup

We need to setup:
* **HuggingFace Token:** In order to make free API calls to HuggingFace Inference API you must have a HF account, the API verifies this by checking your account's token. This token will not be used for anything else.
* **Benchmark context files:** In order to solve the benchmark tasks the agent will need to reference documentation and analyze data which is spread out across multiple files, just like a real Data Analyst would.

### HuggingFace Token Setup

In [ ]:
%pip install smolagents==1.6.0 datasets==3.2.0
# we install also the evaluation logic from the DABstep leaderboard, so we can locally evaluate without having to make a submission
%pip install git+https://git@hf.co/spaces/adyen/DABstep.git@main

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Succ

In [ ]:
import time
import os
import json
import datasets
import pandas as pd
from smolagents import CodeAgent
from smolagents.agents import ActionStep
from smolagents.models import HfApiModel
from huggingface_hub import login, hf_hub_download
from dabstep_benchmark.utils import evaluate


login(new_session=True)

#### Download context files
First we donwload the context files from the [Benchmark's Dataset](https://huggingface.co/datasets/adyen/DABstep) so that our agent can access them.


In [ ]:
CONTEXT_FILENAMES = [
    "data/context/acquirer_countries.csv",
    "data/context/payments-readme.md",
    "data/context/payments.csv",
    "data/context/merchant_category_codes.csv",
    "data/context/fees.json",
    "data/context/merchant_data.json",
    "data/context/manual.md",
]
DATA_DIR = "/tmp/DABstep-data"
for filename in CONTEXT_FILENAMES:
    hf_hub_download(
        repo_id="adyen/DABstep",
        repo_type="dataset",
        filename=filename,
        local_dir=DATA_DIR,
        force_download=True
    )

CONTEXT_FILENAMES = [f"{DATA_DIR}/{filename}" for filename in CONTEXT_FILENAMES]

for file in CONTEXT_FILENAMES:
    if os.path.exists(file):
        print(f"{file} exists.")
    else:
        print(f"{file} does not exist.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


acquirer_countries.csv:   0%|          | 0.00/194 [00:00<?, ?B/s]

payments-readme.md:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

payments.csv:   0%|          | 0.00/23.6M [00:00<?, ?B/s]

merchant_category_codes.csv:   0%|          | 0.00/26.6k [00:00<?, ?B/s]

fees.json:   0%|          | 0.00/531k [00:00<?, ?B/s]

merchant_data.json:   0%|          | 0.00/6.86k [00:00<?, ?B/s]

manual.md:   0%|          | 0.00/22.1k [00:00<?, ?B/s]

/tmp/DABstep-data/data/context/acquirer_countries.csv exists.
/tmp/DABstep-data/data/context/payments-readme.md exists.
/tmp/DABstep-data/data/context/payments.csv exists.
/tmp/DABstep-data/data/context/merchant_category_codes.csv exists.
/tmp/DABstep-data/data/context/fees.json exists.
/tmp/DABstep-data/data/context/merchant_data.json exists.
/tmp/DABstep-data/data/context/manual.md exists.


## Agent

Here we will setup a simple zero-shot prompt for the agent. It has two parts, the general prompt and then a quick outline of which files are available.

In [ ]:
MODEL_ID = "Qwen/Qwen2.5-Coder-32B-Instruct"
HfApiModel(MODEL_ID)

## 3.4 Testing Agent

In [ ]:
MODEL_ID = "Qwen/Qwen2.5-Coder-32B-Instruct"
MAX_STEPS = 7

agent = CodeAgent(
    tools=[],
    model=HfApiModel(MODEL_ID),
    additional_authorized_imports=["numpy", "pandas", "json", "csv", "os", "glob", "markdown"],
    max_steps=MAX_STEPS,
    verbosity_level=3,
)
# give agent power to open files
agent.python_executor.static_tools.update({"open": open})

In [ ]:
PROMPT = """You are an expert data analyst and you will answer factoid questions by loading and referencing the files/documents listed below.
You have these files available:
{context_files}
Don't forget to reference any documentation in the data dir before answering a question.

Here is the question you need to answer:
{question}

Here are the guidelines you must follow when answering the question above:
{guidelines}
"""
question = "What are the unique set of merchants in the payments data?"
question="What is the top country (ip_country) for fraud? A. NL, B. BE, C. ES, D. FR"
guidelines = "Answer with a comma separated list"
guidelines ="Answer must be in the form 'X. Y' where X is the option's letter chosen and Y is the option's country code. If a question does not have a relevant or applicable answer for the task, please respond with 'Not Applicable'"

PROMPT = PROMPT.format(
    context_files=CONTEXT_FILENAMES,
    question=question,
    guidelines=guidelines
)

answer = agent.run(PROMPT)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an expert data analyst and you will answer factoid questions by loading and referencing the             │
│ files/documents listed below.                                                                                   │
│ You have these files available:                                                                                 │
│ ['/tmp/DABstep-data/data/context/acquirer_countries.csv', '/tmp/DABstep-data/data/context/payments-readme.md',  │
│ '/tmp/DABstep-data/data/context/payments.csv', '/tmp/DABstep-data/data/context/merchant_category_codes.csv',    │
│ '/tmp/DABstep-data/data/context/fees.json', '/tmp/DABstep-data/data/context/merchant_data.json',                │
│ '/tmp/DABstep-data/data/context/manual.md']                                                                     │
│ Don't forget to reference any documentation in the data dir before answering a question.                        │
│                                                                                                                 │
│ Here is the question you need to answer:                                                                        │
│ What is the top country (ip_country) for fraud? A. NL, B. BE, C. ES, D. FR                                      │
│                                                                                                                 │
│ Here are the guidelines you must follow when answering the question above:                                      │
│ Answer must be in the form 'X. Y' where X is the option's letter chosen and Y is the option's country code. If  │
│ a question does not have a relevant or applicable answer for the task, please respond with 'Not Applicable'     │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To determine the top country for fraud based on the provided data files, I will first look for any        
documentation that indicates how fraud is tracked or indicated in the data. If no direct fraud label exists, we    
will need to infer it. Since we have a `payments.csv` file, it is likely that the fraud status can be found there. 
                                                                                                                   
First, I will check the contents of the `payments.csv` and `payments-readme.md` files to understand the structure  
of the payments data and any fields indicating fraud.                                                              
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
import pandas as pd                                                                                                
                                                                                                                   
# Load the payments data                                                                                           
payments_df = pd.read_csv('/tmp/DABstep-data/data/context/payments.csv')                                           
                                                                                                                   
# Load the payments readme                                                                                         
with open('/tmp/DABstep-data/data/context/payments-readme.md', 'r') as file:                                       
    payments_readme = file.read()                                                                                  
                                                                                                                   
# Display the payments dataframe and the readme content                                                            
print(payments_df.head())                                                                                          
print("="*80)                                                                                                      
print(payments_readme)                                                                                             
```                                                                                                                
<end_code>                                                                                                         

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Load the payments data                                                                                         
  payments_df = pd.read_csv('/tmp/DABstep-data/data/context/payments.csv')                                         
                                                                                                                   
  # Load the payments readme                                                                                       
  with open('/tmp/DABstep-data/data/context/payments-readme.md', 'r') as file:                                     
      payments_readme = file.read()                                                                                
                                                                                                                   
  # Display the payments dataframe and the readme content                                                          
  print(payments_df.head())                                                                                        
  print("="*80)                                                                                                    
  print(payments_readme)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
   psp_reference               merchant card_scheme  year  hour_of_day  \
0    20034594130         Crossfit_Hanna      NexPay  2023           16   
1    36926127356         Crossfit_Hanna      NexPay  2023           23   
2    31114608278  Belles_cookbook_store  GlobalCard  2023            4   
3    68442235288         Crossfit_Hanna      NexPay  2023            3   
4    81404384199         Crossfit_Hanna      NexPay  2023           17   

   minute_of_hour  day_of_year  is_credit  eur_amount ip_country  ...  \
0              21           12      False      151.74         SE  ...   
1              58           75      False       45.70         NL  ...   
2              30           96      False       14.11         NL  ...   
3               5           77       True      238.42         LU  ...   
4              30           83      False       67.13         NL  ...   

  device_type              ip_address           email_address  \
0     Windows  pKPYzJqqwB8TdpY0jiAeQw  0AKXyaTjW7H4m1hOWmOKBQ   
1       Linux  uzUknOkIqExYsWv4X14GUg  _Gm8at1k2ojYAM_wSEptNw   
2       MacOS  3VO1v_RndDg6jzEiPjfvoQ                     NaN   
3         iOS  3qbuXGoFldniCC6r1X8K0Q  5VW_2O6ku_0p_fLLwuC1vw   
4     Windows  9WMJJdgtop6jkkyerxMvuQ  Alb1iUIxIqlW8YUeYVGTzg   

              card_number shopper_interaction card_bin  \
0  uRofX46FuLUrSOTz8AW5UQ           Ecommerce     4802   
1  6vqQ89zfCeFk6s4VOoWZFQ           Ecommerce     4920   
2  EmxSN8-GXQw3RG_2v7xKxQ           Ecommerce     4571   
3  wG2VTvj2TfVG-NRDzifMHw           Ecommerce     4017   
4  0khzuCj7aQ1e51S5vWR8gg           Ecommerce     4532   

   has_fraudulent_dispute  is_refused_by_adyen  aci acquirer_country  
0                   False                False    F               NL  
1                   False                False    F               NL  
2                   False                False    F               US  
3                   False                False    D               NL  
4                   False                False    F               NL  

[5 rows x 21 columns]
================================================================================
This is documentation for the payments.csv dataset


- **Description**: Synthetic dataset of payment transactions processed by the Payments Processor.
- **Columns**:
  - `psp_reference`: Unique payment identifier (ID).
  - `merchant`: Merchant name (Categorical), eg Starbucks or Netflix*.
  - `card_scheme`: Card Scheme used (Categorical) - *[MasterCard, Visa, Amex, Other]*.
  - `year`: Payment initiation year (Numeric).
  - `hour_of_day`: Payment initiation hour (Numeric).
  - `minute_of_hour`: Payment initiation minute (Numeric).
  - `day_of_year`: Day of the year of payment initiation (Numeric).
  - `is_credit`: Credit or Debit card indicator (Categorical).
  - `eur_amount`: Payment amount in euro (Numeric).
  - `ip_country`: The country the shopper was in at time of transaction (determined by IP address) (Categorical) - 
*[SE, NL, LU, IT, BE, FR, GR, ES]*.
  - `issuing_country`: Card-issuing country (Categorical) - *[SE, NL, LU, IT, BE, FR, GR, ES]*.
  - `device_type`: Device type used (Categorical) - *[Windows, Linux, MacOS, iOS, Android, Other]*.
  - `ip_address`: Hashed shopper's IP (ID).
  - `email_address`: Hashed shopper's email (ID).
  - `card_number`: Hashed card number (ID).
  - `shopper_interaction`: Payment method (Categorical) - *[Ecommerce, POS]*. POS means an in-person or in-store 
transaction.
  - `card_bin`: Bank Identification Number (ID).
  - `has_fraudulent_dispute`: Indicator of fraudulent dispute from issuing bank (Boolean).
  - `is_refused_by_adyen`: Adyen refusal indicator (Boolean).
  - `aci`: Authorization Characteristics Indicator (Categorical).
  - `acquirer_country`: The location (country) of the acquiring bank (Categorical) - *[SE, NL, LU, IT, BE, FR, GR, 
ES]*.

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: The top country with the highest number of fraudulent disputes is NL. Based on the provided options and   
the result of our analysis, the correct answer is "A. NL".                                                         
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
final_answer("A. NL")                                                                                              
```                                                                                                                
<end_code>                                                                                                         

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("A. NL")                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: A. NL

[Step 2: Duration 0.12 seconds| Input tokens: 10,478 | Output tokens: 526]

In [ ]:
# You can inspect the steps taken by the agent by doing this
def clean_reasoning_trace(trace: list) -> list:
  for step in trace:
      # Remove memory from logs to make them more compact.
      if hasattr(step, "memory"):
          step.memory = None
      if isinstance(step, ActionStep):
          step.agent_memory = None
  return trace



for step in clean_reasoning_trace(agent.logs):
    print(step)

SystemPromptStep(system_prompt='You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important information you will then need.\nThese print outputs will then appear in the \'Observation:\' field, which will be available as input for the next step.\nIn the end you have to return a final answer using t

## 3.5 Warmup Agent

We encourage to first get a good performing agent on this smaller dev split of the benchmark since running against the full benchmark can be very costly both in time and compute.

Once we are confident with our agent, we can try to create a submission to the leaderboard using the complete dataset of tasks


In [ ]:
def run_benchmark(dataset: datasets.Dataset, agent: CodeAgent) -> list[dict]:
  agent_answers = []
  for task in dataset:
    tid = task['task_id']

    prompt = PROMPT.format(
      context_files=CONTEXT_FILENAMES,
      question=task['question'],
      guidelines=task['guidelines']
    )

    answer = agent.run(prompt)

    task_answer = {
        "task_id": str(tid),
        "agent_answer": str(answer),
        "reasoning_trace": str(clean_reasoning_trace(agent.logs))
    }

    agent_answers.append(task_answer)

  return agent_answers

In [ ]:
MAX_STEPS = 7
MODEL_ID = "Qwen/Qwen2.5-Coder-32B-Instruct"

agent = CodeAgent(
    tools=[], # evaluating zero-shot capabilities of the model when writing code
    model=HfApiModel(MODEL_ID),
    additional_authorized_imports=["numpy", "pandas", "json", "csv", "os", "glob", "markdown"],
    max_steps=MAX_STEPS,
    verbose=False
)

# give agent power to open files
agent.python_executor.static_tools.update({"open": open})

In [ ]:
PROMPT = """You are an expert data analyst and you will answer factoid questions by loading and referencing the files/documents listed below.
You have these files available:
{context_files}
Don't forget to reference any documentation in the data dir before answering a question.

Here is the question you need to answer:
{question}

Here are the guidelines you must follow when answering the question above:
{guidelines}
"""

SPLIT = "dev"
MAX_TASKS = 3 # It takes ~20 min to run the agent on all the dev set, so lets start just with the 2 first tasks
RUN_ID = int(time.time())


# load dataset from Hub
dev_task_dataset = datasets.load_dataset("adyen/DABstep", name="tasks", split=f"{SPLIT}[:{MAX_TASKS}]")
%time agent_answers = run_benchmark(dataset=dev_task_dataset, agent=agent)


### Evaluation

In [ ]:
# Lets visualize the answers from our agent
pd.DataFrame(agent_answers)

In [ ]:
# Now we evaluate the answers
agent_answers = pd.DataFrame(agent_answers)
tasks_df = dev_task_dataset.to_pandas()
task_scores = evaluate(agent_answers=agent_answers, tasks_with_gt=tasks_df)

In [ ]:
# Inspect scores
task_scores = pd.DataFrame(task_scores)
task_scores["correct_answer"] = tasks_df["answer"]
task_scores["question"] = tasks_df["question"]
task_scores

## Final submission
Now we run the agent against the full benchmark and create a submission file to submit to the leaderboard


⚠️ *THIS WILL TAKE A LONG TIME IF YOU RUN THE FOR THE FULL BENCHMARK (450 tasks)* ⚠️

In [ ]:
from pathlib import Path

# Crate dir to store agent run results
RUNS_DIR = Path().resolve() / "runs"
RUNS_DIR.mkdir(parents=True, exist_ok=True)

def write_jsonl(data: list[dict], filepath: Path) -> None:
    """Write a list of dictionaries to a JSONL file."""
    # Ensure the directory exists
    filepath.parent.mkdir(parents=True, exist_ok=True)

    with open(filepath, "w") as file:
        for entry in data:
            file.write(json.dumps(entry) + "\n")

In [ ]:
PROMPT = """You are an expert data analyst and you will answer factoid questions by loading and referencing the files/documents listed below.
You have these files available:
{context_files}

Here is the question you need to answer:
{question}

Here are the guidelines you must follow when answering the question above:
{guidelines}
"""

# now we run all dataset
SPLIT = "default"
RUN_ID = int(time.time())


# load dataset from Hub
benchmark_task_dataset = datasets.load_dataset("adyen/DABstep", name="tasks", split=SPLIT)
%time agent_answers = run_benchmark(dataset=benchmark_task_dataset, agent=agent)

write_jsonl(agent_answers, RUNS_DIR / f"{RUN_ID}.jsonl")

In [ ]:
# Lets visualize the answers from our agent
pd.DataFrame(agent_answers)

## Submission to Leaderboard

Your submission file should be saved in the `runs` folder of this directory.

Now is the fun part, were you submit it to the Leaderboard and see your score!

The leaderboard is here: https://huggingface.co/spaces/adyen/DABstep


### Analysis your submission scores

Once you have made a submission you can inspect the scores of your submission like these. Task scores are more granular task-level scores.

In [ ]:
submissions_dataset = datasets.load_dataset("adyen/DABstep", name="submissions", split="default")
task_scores_dataset = datasets.load_dataset("adyen/DABstep", name="task_scores", split="default")

AGENT_NAME = "Claude 3.5 Sonnet ReACT Baseline" # the agent name given in the submission form
ORGANISATION = "Adyen" # the organization name given in the submission form
SUBMISSION_ID = f"{ORGANISATION}-{AGENT_NAME}"

submissions_dataset_df = submissions_dataset.filter(lambda row: row["submission_id"] == SUBMISSION_ID).to_pandas()
task_scores_dataset_df = task_scores_dataset.filter(lambda row: row["submission_id"] == SUBMISSION_ID).to_pandas()

In [ ]:
submissions_dataset_df

In [ ]:
task_scores_dataset_df

## Next Steps

This is a few-shot ReAct prompt by deafult

Some things you can try next:

* do an error analysis and see why you failed certain questions
* tweaking the zero-shot prompt
* try a few-shot prompt
* try different agentic workflows like: CoT